In [8]:
import os
import re
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import gradio as gr
import logging
import random
from datetime import datetime
import sqlite3
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import speech_recognition as sr
from gtts import gTTS
import uuid
import base64
import tempfile
from io import BytesIO
import threading
import time
import cv2

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
except:
    logger.warning("NLTK download failed. Make sure NLTK is properly installed.")

class LoubbyDocumentation:
    """Class to handle Loubby platform documentation"""
    
    def __init__(self, docs_path="loubby_docs.txt"):
        """Initialize the documentation handler"""
        self.docs_path = docs_path
        self.sections = {}
        self.raw_text = self._load_documentation()
        self.process_documentation()
        
    def _load_documentation(self) -> str:
        """Load documentation from file or create it if it doesn't exist"""
        if not os.path.exists(self.docs_path):
            logger.info(f"Documentation file not found at {self.docs_path}. Creating it...")
            self._create_documentation_file()
        
        with open(self.docs_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def _create_documentation_file(self):
        """Create the documentation file with Loubby platform information"""
        documentation_text = """# Loubby Platform Navigation Guide

## Recruitment Portal

### 1. Profile Setup
#### 1.1 Creating Your Account
To create your account on Loubby's recruitment portal:
1. Navigate to loubby.ai/recruitment
2. Click the "Sign Up" button in the top right corner
3. Enter your email address and create a password
4. Verify your email through the link sent to your inbox
5. Complete your basic profile information (name, contact details)

#### 1.2 Resume Upload
To upload your resume to your Loubby profile:
1. Log in to your account
2. Navigate to "My Profile" from the dashboard menu
3. Scroll to the "Resume" section
4. Click "Upload Resume" button
5. Select your resume file (supported formats: PDF, DOCX, RTF)
6. Click "Save Changes" to confirm upload
7. Verify that your resume appears in the preview section

Tip: For best results, use a PDF format that is ATS-friendly

#### 1.3 Skills Assessment
To complete the skills assessment:
1. From the dashboard, select "Skills Assessment" tab
2. Choose the relevant skill categories for your profile
3. For each category, click "Start Assessment"
4. Complete the assessment questions within the allotted time
5. Submit your answers using the "Submit" button
6. View your results and skill ratings on completion

### 2. Job Search and Application

#### 2.1 Searching for Positions
To search for open positions on Loubby:
1. Click the "Jobs" tab in the main navigation
2. Use the search bar to enter keywords, job titles, or company names
3. Filter results using the sidebar options:
   - Location (remote, on-site, hybrid)
   - Experience level
   - Industry
   - Salary range
   - Posted date
4. Sort results by relevance, date, or salary using the dropdown menu
5. Save searches by clicking "Save This Search" for future reference

#### 2.2 Applying for a Position
To apply for a position:
1. From the job listing, click "Apply Now"
2. Review your profile information for completeness
3. Complete any additional application questions
4. Upload any position-specific documents if requested
5. Review all information for accuracy
6. Click "Submit Application"
7. Confirm submission in the popup window

#### 2.3 Tracking Applications
To track your submitted applications:
1. Navigate to "My Applications" in the dashboard
2. View all applications and their current status
3. Filter by status (submitted, in review, interview scheduled, etc.)
4. Click on any application to view details
5. Check for any action items or requested information

### 3. Interview Process

#### 3.1 Scheduling Interviews
When you receive an interview request:
1. You'll receive an email notification and an in-platform notification
2. Navigate to "My Applications" in the dashboard
3. Find the application with the interview request
4. Click on "Schedule Interview"
5. Select from available time slots in the calendar interface
6. Confirm your selection
7. Add to your personal calendar using the calendar integration options

#### 3.2 Virtual Interview Room
To access the virtual interview room:
1. 15 minutes before your scheduled interview, log in to Loubby
2. Go to "My Applications" in the dashboard
3. Find the relevant application
4. Click "Join Interview" (button becomes active 15 minutes before start time)
5. Test your audio and video in the pre-meeting room
6. Click "Join Now" when ready
7. If you experience technical difficulties, use the support chat in the bottom right

## Employee Portal

### 1. Onboarding

#### 1.1 Accessing the Employee Portal
After accepting a job offer:
1. You'll receive an email with login credentials for the employee portal
2. Navigate to loubby.ai/employee
3. Enter your credentials from the email
4. Set up two-factor authentication if prompted
5. Create a new password that meets security requirements

#### 1.2 Completing Onboarding Documents
To complete necessary onboarding paperwork:
1. From the employee dashboard, select "Onboarding" tab
2. View the list of required documents
3. Click "Complete" next to each document
4. Fill out all required fields
5. Use the e-signature tool where required
6. Click "Submit" for each completed document
7. Track completion progress in the onboarding checklist

#### 1.3 Training Modules
To access and complete training modules:
1. Navigate to "Training" in the main menu
2. View assigned training modules and their deadlines
3. Click on a module to begin
4. Complete all sections of the module
5. Take the assessment quiz at the end
6. Achieve the minimum required score to mark as complete
7. View your training transcript in the "My Learning" section

### 2. Course Synchronization

#### 2.1 Connecting External Courses
To sync external courses with the Loubby platform:
1. From the dashboard, navigate to "Education & Courses"
2. Select "Connect External Course" button
3. Choose from the list of supported learning platforms
4. Log in to your account on the selected platform
5. Authorize the connection when prompted
6. Select which courses to sync with Loubby
7. Click "Confirm" to complete the integration

#### 2.2 Viewing Course Progress
To monitor your progress across all courses:
1. Go to "My Learning" in the main navigation
2. View the dashboard showing all courses and completion percentages
3. Filter by course status, start date, or provider
4. Click on any course to view detailed progress
5. See upcoming assignments and deadlines
6. View instructor feedback on completed assignments

#### 2.3 Submitting Assignments
To submit assignments through Loubby:
1. Navigate to "My Learning" > "Assignments"
2. Find the assignment you want to submit
3. Click "Submit Assignment"
4. Upload required files or enter text submission
5. Add any comments for the instructor
6. Review your submission
7. Click "Submit" to finalize

### 3. Performance Tracking

#### 3.1 Setting Goals
To set and track professional goals:
1. From the employee dashboard, select "Performance" tab
2. Click "My Goals" in the submenu
3. Select "Add New Goal"
4. Define your goal with the SMART criteria
5. Set target date and key milestones
6. Link to relevant skills or competencies
7. Submit for manager approval

#### 3.2 Feedback and Reviews
To access performance feedback:
1. Navigate to "Performance" > "Feedback"
2. View all feedback received from peers and supervisors
3. Filter by date, project, or feedback type
4. For formal reviews, select "Reviews" from the submenu
5. View scheduled and past review cycles
6. Complete self-assessment when prompted
7. Review manager feedback and ratings

## Troubleshooting

### Common Issues

#### Password Reset
If you need to reset your password:
1. On the login page, click "Forgot Password"
2. Enter the email associated your account
3. Check your email for a reset link
4. Click the link and follow instructions to create a new password
5. If you don't receive the email, check spam folder or click "Resend Email"

#### Browser Compatibility
Loubby works best with:
- Chrome (version 90+)
- Firefox (version 88+)
- Safari (version 14+)
- Edge (version 90+)

If experiencing issues:
1. Update your browser to the latest version
2. Clear browser cache and cookies
3. Disable extensions that might interfere
4. Try an alternate supported browser

#### Mobile Access
To access Loubby on mobile devices:
1. Use the responsive web version on any mobile browser
2. Download the Loubby mobile app from App Store or Google Play
3. Log in with the same credentials as the web version
4. Enable notifications for interview alerts
5. Note that some advanced features may require desktop access

### Getting Help

#### Live Support
To access live support:
1. Click the "Help" icon in the bottom right corner
2. Select "Chat with Support"
3. Describe your issue in detail
4. Support is available Monday-Friday, 9am-6pm EST

#### Knowledge Base
To search the help documentation:
1. Click "Help" in the main navigation
2. Use the search bar to find relevant articles
3. Browse by category using the sidebar
4. Rate articles to help improve documentation
"""
        with open(self.docs_path, 'w', encoding='utf-8') as f:
            f.write(documentation_text)
        
        logger.info(f"Documentation file created at {self.docs_path}")
    
    def process_documentation(self):
        """Process the documentation into sections"""
        # Split by main sections using regex
        section_pattern = r'(#+)\s+(.+)'
        current_section = {"title": "Root", "content": "", "level": 0, "subsections": []}
        section_stack = [current_section]
        
        for line in self.raw_text.split('\n'):
            match = re.match(section_pattern, line)
            if match:
                level = len(match.group(1))
                title = match.group(2).strip()
                
                new_section = {"title": title, "content": "", "level": level, "subsections": []}
                
                # Pop from stack until we find the parent section
                while level <= section_stack[-1]["level"]:
                    section_stack.pop()
                
                # Add new section as subsection of parent
                section_stack[-1]["subsections"].append(new_section)
                
                # Push new section to stack
                section_stack.append(new_section)
            else:
                # Add content to current section
                if section_stack[-1]["content"]:
                    section_stack[-1]["content"] += "\n" + line
                else:
                    section_stack[-1]["content"] += line
        
        self.sections = section_stack[0]
        logger.info("Documentation processed successfully")

    def get_flattened_sections(self) -> List[Dict]:
        """Flatten the hierarchical sections into a list with full paths"""
        flattened = []
        
        def traverse(section, path=""):
            current_path = f"{path} > {section['title']}" if path else section['title']
            
            # Add current section
            if section['content'].strip():
                flattened.append({
                    "path": current_path,
                    "title": section['title'],
                    "content": section['content'],
                    "level": section['level']
                })
            
            # Process subsections
            for subsection in section['subsections']:
                traverse(subsection, current_path)
        
        traverse(self.sections)
        return flattened

class DocumentChunker:
    """Class to chunk documentation for efficient retrieval"""
    
    def __init__(self, chunk_size=200, chunk_overlap=50):
        """Initialize the document chunker"""
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
    
    def chunk_document(self, sections: List[Dict]) -> List[Dict]:
        """Split sections into smaller chunks for retrieval"""
        chunked_sections = []
        
        for section in sections:
            # Combine section info with content for context
            full_text = f"{section['path']}\n\n{section['content']}"
            
            # Split text into chunks
            chunks = self.text_splitter.split_text(full_text)
            
            # Create dictionary entries for each chunk
            for i, chunk_text in enumerate(chunks):
                chunked_sections.append({
                    "id": f"{section['path']}_{i}",
                    "path": section['path'],
                    "title": section['title'],
                    "chunk_index": i,
                    "content": chunk_text,
                    "level": section['level']
                })
        
        logger.info(f"Created {len(chunked_sections)} chunks from {len(sections)} sections")
        return chunked_sections

class VectorDatabase:
    """Class to manage vector embeddings and similarity search"""
    
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """Initialize the vector database"""
        self.model_name = model_name
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
        self.vectorstore = None
        self.chunk_data = None
    
    def create_vectors(self, chunked_sections: List[Dict]):
        """Create vector embeddings for document chunks"""
        # Extract texts and metadata
        texts = [chunk["content"] for chunk in chunked_sections]
        metadatas = [{
            "id": chunk["id"],
            "path": chunk["path"],
            "title": chunk["title"],
            "chunk_index": chunk["chunk_index"],
            "level": chunk["level"]
        } for chunk in chunked_sections]
        
        # Create FAISS index
        self.vectorstore = FAISS.from_texts(
            texts=texts,
            embedding=self.embeddings,
            metadatas=metadatas
        )
        
        # Store the original chunk data for reference
        self.chunk_data = {chunk["id"]: chunk for chunk in chunked_sections]}
        
        logger.info(f"Created vector database with {len(texts)} documents")
        return self.vectorstore
    
    def save_vectors(self, path="loubby_vectors"):
        """Save vector database to disk"""
        if self.vectorstore:
            self.vectorstore.save_local(path)
            # Save chunk data separately
            with open(f"{path}_chunks.json", 'w', encoding='utf-8') as f:
                json.dump(self.chunk_data, f)
            logger.info(f"Vector database saved to {path}")
        else:
            logger.error("No vector database to save")
    
    def load_vectors(self, path="loubby_vectors"):
        """Load vector database from disk"""
        if os.path.exists(path):
            try:
                self.vectorstore = FAISS.load_local(path, self.embeddings, allow_dangerous_deserialization=True)
                # Load chunk data
                with open(f"{path}_chunks.json", 'r', encoding='utf-8') as f:
                    self.chunk_data = json.load(f)
                logger.info(f"Vector database loaded from {path}")
                return True
            except Exception as e:
                logger.error(f"Error loading vector database: {e}")
                return False
        else:
            logger.warning(f"No vector database found at {path}")
            return False
    
    def similarity_search(self, query: str, k=3) -> List[Dict]:
        """Search for most similar chunks to the query"""
        if not self.vectorstore:
            logger.error("Vector database not initialized")
            return []
        
        results = self.vectorstore.similarity_search_with_score(query, k=k)
        
        # Format results
        formatted_results = []
        for doc, score in results:
            chunk_id = doc.metadata["id"]
            formatted_results.append({
                "id": chunk_id,
                "path": doc.metadata["path"],
                "title": doc.metadata["title"],
                "content": doc.page_content,
                "similarity": float(score),
                "chunk_index": doc.metadata["chunk_index"],
                "level": doc.metadata["level"]
            })
        
        return formatted_results

class FeedbackSystem:
    """System to collect and use user feedback to improve responses"""
    
    def __init__(self, db_path="loubby_feedback.db"):
        """Initialize the feedback system"""
        self.db_path = db_path
        self._init_database()
    
    def _init_database(self):
        """Initialize the SQLite database for feedback"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Create feedback table if it doesn't exist
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS feedback (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    query TEXT NOT NULL,
                    response TEXT NOT NULL,
                    rating INTEGER NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    user_comment TEXT
                )
            ''')
            
            conn.commit()
            conn.close()
            logger.info(f"Feedback database initialized at {self.db_path}")
        except Exception as e:
            logger.error(f"Error initializing feedback database: {e}")
    
    def record_feedback(self, query: str, response: str, rating: int, user_comment: str = ""):
        """Record user feedback on a response"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute(
                "INSERT INTO feedback (query, response, rating, user_comment) VALUES (?, ?, ?, ?)",
                (query, response, rating, user_comment)
            )
            
            conn.commit()
            conn.close()
            logger.info(f"Feedback recorded: Rating {rating} for query '{query[:30]}...'")
            return True
        except Exception as e:
            logger.error(f"Error recording feedback: {e}")
            return False
    
    def get_historical_feedback(self, limit=100) -> List[Dict]:
        """Get historical feedback data"""
        try:
            conn = sqlite3.connect(self.db_path)
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            cursor.execute(
                "SELECT * FROM feedback ORDER BY timestamp DESC LIMIT ?",
                (limit,)
            )
            
            # Convert to list of dictionaries
            results = [dict(row) for row in cursor.fetchall()]
            
            conn.close()
            return results
        except Exception as e:
            logger.error(f"Error fetching historical feedback: {e}")
            return []
    
    def get_feedback_statistics(self) -> Dict:
        """Get statistics about feedback ratings"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Get average rating
            cursor.execute("SELECT AVG(rrating) FROM feedback")
            avg_rating = cursor.fetchone()[0] or 0
            
            # Get rating distribution
            cursor.execute(
                "SELECT rating, COUNT(*) FROM feedback GROUP BY rating ORDER BY rating"
            )
            rating_counts = dict(cursor.fetchall())
            
            # Get total feedback count
            cursor.execute("SELECT COUNT(*) FROM feedback")
            total_count = cursor.fetchone()[0]
            
            conn.close()
            
            return {
                "average_rating": round(avg_rating, 2),
                "rating_distribution": rating_counts,
                "total_count": total_count
            }
        except Exception as e:
            logger.error(f"Error fetching feedback statistics: {e}")
            return {
                "average_rating": 0,
                "rating_distribution": {},
                "total_count": 0
            }

class LanguageProcessor:
    """Class for handling natural language processing tasks"""
    
    def __init__(self, model_path="google/flan-t5-base"):
        """Initialize the language processor"""
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
            logger.info(f"Language processor initialized with model {model_path}")
            
            # Load language detection model
            self.lang_detector = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
            self.language_map = {
                "en": "English",
                "es": "Spanish", 
                "fr": "French",
                "de": "German",
                "zh": "Chinese",
                "ja": "Japanese",
                "ru": "Russian",
                "ar": "Arabic",
                "hi": "Hindi",
                "pt": "Portuguese",
                "it": "Italian",
                "ko": "Korean"
            }
        except Exception as e:
            logger.error(f"Error initializing language processor: {e}")
            self.tokenizer = None
            self.model = None
    
    def detect_language(self, text: str) -> str:
        """Detect the language of input text using a multilingual sentence transformer"""
        if not text.strip():
            return "en"  # Default to English for empty text
        
        try:
            # Get embeddings for the text
            text_embedding = self.lang_detector.encode(text)
            
            # Compare with embeddings of language markers
            language_scores = {}
            for lang, lang_name in self.language_map.items():
                # Create a simple sentence in the target language
                lang_marker = f"This is a sentence in {lang_name}."
                lang_embedding = self.lang_detector.encode(lang_marker)
                
                # Calculate cosine similarity
                similarity = cosine_similarity([text_embedding], [lang_embedding])[0][0]
                language_scores[lang] = similarity
            
            # Return the language with the highest similarity
            detected_lang = max(language_scores.items(), key=lambda x: x[1])[0]
            logger.info(f"Detected language: {detected_lang} with score {language_scores[detected_lang]:.2f}")
            
            return detected_lang
        except Exception as e:
            logger.error(f"Error detecting language: {e}")
            return "en"  # Default to English on error
    
    def translate_text(self, text: str, target_language: str = "en") -> str:
        """Simple translation function (placeholder for actual translation)"""
        # Note: In a real implementation, you would integrate with a translation API
        # Since we're omitting external API dependencies for this cloud environment, 
        # we'll return the original text with a note
        
        detected_language = self.detect_language(text)
        
        # Skip translation if already in target language
        if detected_language == target_language:
            return text
        
        logger.info(f"Translation requested from {detected_language} to {target_language}")
        
        # In a real implementation, you would call a translation service here
        # For now, just return the original text
        return text
    
    def generate_response(self, query: str, retrieved_chunks: List[Dict]) -> str:
        """Generate a comprehensive response based on the retrieved chunks"""
        if not self.model or not self.tokenizer:
            return "Language model not properly initialized."
        
        try:
            # Create a context from retrieved chunks
            context = "\n\n".join([chunk["content"] for chunk in retrieved_chunks])
            
            # Build a prompt
            prompt = f"""
            CONTEXT: {context}
            
            QUERY: {query}
            
            Based on the above context, provide a clear and concise answer to the query about navigating the Loubby platform.
            """
            
            # Generate response
            inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=512,
                min_length=64,
                num_beams=4,
                no_repeat_ngram_size=3,
                early_stopping=True
            )
            
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return response
        except Exception as e:
            logger.error(f"Error generating response: {e}")
            
            # Fallback response method
            most_relevant_chunk = retrieved_chunks[0] if retrieved_chunks else None
            if most_relevant_chunk:
                return f"Based on the Loubby documentation, here's guidance on your question:\n\n{most_relevant_chunk['content']}"
            else:
                return "I'm unable to generate a specific response at the moment. Please try rephrasing your question about the Loubby platform."

class VoiceInterface:
    """Class to handle voice input and output using Gradio components"""
    
    def __init__(self):
        """Initialize the voice interface"""
        logger.info("Voice interface initialized using Gradio components")
    
    def process_audio(self, audio):
        """Process audio input and return text"""
        try:
            if audio is None:
                return ""
            
            # Use Gradio's built-in audio processing
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio) as source:
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data)
                return text
        except Exception as e:
            logger.error(f"Error processing audio: {e}")
            return ""
    
    def generate_audio(self, text, language="en"):
        """Generate audio from text with language support"""
        try:
            if not text.strip():
                return None
            
            tts = gTTS(text=text, lang=language)
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as f:
                tts.save(f.name)
                return f.name
        except Exception as e:
            logger.error(f"Error generating audio: {e}")
            return None

class VideoInterface:
    """Class to handle video communication in cloud environment"""
    
    def __init__(self):
        """Initialize the video interface with dummy video for cloud environments"""
        self.is_running = False
        self.thread = None
        self.frame = None
        self.user_face_detected = False
        self.animation_frame = 0
        self.animation_speed = 0.15
        self.last_animation_time = time.time()
        
        # Create avatars since we can't use real camera
        self.user_avatar = self._create_user_avatar()
        self.assistant_avatar = self._create_assistant_avatar()
        
        logger.info("Video interface initialized with virtual avatars for cloud environment")
    
    def _create_user_avatar(self):
        """Create a simple avatar for the user in cloud environments"""
        avatar = np.zeros((240, 320, 3), dtype=np.uint8)
        # Set background color
        avatar[:, :] = (50, 50, 50)
        # Draw a simple circle face
        cv2.circle(avatar, (160, 120), 80, (100, 100, 220), -1)
        # Draw eyes
        cv2.circle(avatar, (130, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (190, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (130, 100), 7, (0, 0, 0), -1)
        cv2.circle(avatar, (190, 100), 7, (0, 0, 0), -1)
        # Draw mouth
        cv2.ellipse(avatar, (160, 140), (40, 20), 0, 0, 180, (0, 0, 0), 3)
        
        # Add text label
        cv2.putText(avatar, "User (virtual)", (80, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(avatar, "Cloud Environment", (70, 230), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        
        return avatar
    
    def _create_assistant_avatar(self):
        """Create a simple avatar for the assistant"""
        avatar = np.zeros((240, 320, 3), dtype=np.uint8)
        # Set background color
        avatar[:, :] = (30, 30, 30)
        # Draw a simple circle face
        cv2.circle(avatar, (160, 120), 80, (70, 130, 180), -1)
        # Draw eyes
        cv2.circle(avatar, (130, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (190, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (130, 100), 7, (0, 0, 0), -1)
        cv2.circle(avatar, (190, 100), 7, (0, 0, 0), -1)
        # Draw mouth (will be animated)
        cv2.ellipse(avatar, (160, 140), (40, 15), 0, 0, 180, (0, 0, 0), 3)
        
        # Add text label
        cv2.putText(avatar, "Loubby Assistant", (100, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        return avatar
    
    def _update_avatar_animation(self):
        """Update the avatar's animation"""
        current_time = time.time()
        if current_time - self.last_animation_time > self.animation_speed:
            # Create a copy of the base avatar
            animated_avatar = self.assistant_avatar.copy()
            
            # Animate the mouth based on speaking state
            mouth_height = 10 + 10 * np.sin(self.animation_frame)
            cv2.ellipse(animated_avatar, (160, 140), (40, int(mouth_height)), 0, 0, 180, (0, 0, 0), 3)
            
            self.animation_frame += 0.5
            self.last_animation_time = current_time
            
            return animated_avatar
        return self.assistant_avatar.copy()
    
    def start_video(self):
        """Start the virtual video session"""
        if self.is_running:
            return True
        
        self.is_running = True
        
        # Start animation thread
        self.thread = threading.Thread(target=self._animate_avatars)
        self.thread.daemon = True
        self.thread.start()
        
        logger.info("Virtual video avatars started for cloud environment")
        return True
    
    def stop_video(self):
        """Stop the virtual video session"""
        self.is_running = False
        
        if self.thread:
            self.thread.join(timeout=1.0)
            self.thread = None
        
        logger.info("Virtual video avatars stopped")
        return True
    
    def _animate_avatars(self):
        """Animate the avatars in a continuous loop"""
        while self.is_running:
            # Update the assistant avatar animation
            self._update_avatar_animation()
            time.sleep(0.05)
    
    def get_user_frame(self):
        """Get the user's avatar frame"""
        return self.user_avatar
    
    def get_assistant_frame(self):
        """Get the assistant's animated avatar frame"""
        return self._update_avatar_animation()
    
    def generate_combined_frame(self):
        """Generate a combined frame with user and assistant avatars"""
        user_frame = self.get_user_frame()
        assistant_frame = self.get_assistant_frame()
        
        # Create a combined frame (side by side)
        combined = np.zeros((240, 640, 3), dtype=np.uint8)
        combined[:, :320] = user_frame
        combined[:, 320:] = assistant_frame
        
        # Add a separating line
        cv2.line(combined, (320, 0), (320, 240), (100, 100, 100), 2)
        
        # Add a status indicator showing this is a cloud environment
        cv2.rectangle(combined, (180, 10), (460, 30), (0, 0, 100), -1)
        cv2.putText(combined, "Cloud Environment - Virtual Video", (190, 25), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return combined
    
    def process_video_call(self, text_message):
        """Process a simulated video call with the assistant"""
        if not self.is_running:
            return "Please start the video call first.", self._create_assistant_avatar()
        
        # Generate a combined frame with user and assistant
        combined_frame = self.generate_combined_frame()
        
        if not text_message:
            return "I'm here and ready to help! Type a message to ask about the Loubby platform.", combined_frame
        
        return f"Processing: '{text_message}'", combined_frame

class LoubbyAssistant:
    """Main assistant class that integrates all components"""
    
    def __init__(self, load_existing=True):
        """Initialize the Loubby Assistant"""
        self.documentation = LoubbyDocumentation()
        self.chunker = DocumentChunker(chunk_size=300, chunk_overlap=100)
        self.vector_db = VectorDatabase()
        self.feedback_system = FeedbackSystem()
        self.language_processor = LanguageProcessor()
        self.voice_interface = VoiceInterface()
        self.video_interface = VideoInterface()
        
        # Initialize vector database
        if load_existing and self.vector_db.load_vectors():
            logger.info("Using existing vector database")
        else:
            logger.info("Creating new vector database")
            sections = self.documentation.get_flattened_sections()
            chunks = self.chunker.chunk_document(sections)
            self.vector_db.create_vectors(chunks)
            self.vector_db.save_vectors()
        
        # Track conversation history
        self.conversation_history = []
        
        # Map of language codes to full names for internal use
        self.language_map = {
            "en": "English",
            "es": "Spanish", 
            "fr": "French",
            "de": "German",
            "zh": "Chinese",
            "ja": "Japanese",
            "ru": "Russian",
            "ar": "Arabic",
            "hi": "Hindi",
            "pt": "Portuguese",
            "it": "Italian",
            "ko": "Korean"
        }
        
        logger.info("Loubby Assistant initialized and ready")
    
    async def process_query(self, query: str, use_voice=False) -> Dict:
        """Process a user query and generate a response with language detection"""
        start_time = datetime.now()
        
        # Detect language (using our improved detection method)
        detected_language = self.language_processor.detect_language(query)
        logger.info(f"Detected language: {detected_language}")
        
        translated_query = query
        
        # Translate to English for processing if not already in English
        if detected_language != "en":
            translated_query = self.language_processor.translate_text(query, target_language="en")
            logger.info(f"Translated query from {detected_language} to English: {translated_query}")
        
        # Search for relevant chunks
        retrieved_chunks = self.vector_db.similarity_search(translated_query, k=3)
        
        # Generate response in English
        response_en = ""
        if retrieved_chunks:
            response_en = self.language_processor.generate_response(translated_query, retrieved_chunks)
        else:
            response_en = "I'm sorry, I couldn't find specific information about that in the Loubby documentation. Could you please rephrase your question about using the Loubby platform?"
        
        # Translate response back to original language if needed
        response = response_en
        if detected_language != "en":
            try:
               response = self.language_processor.translate_text(response_en, target_language=detected_language)
               logger.info(f"Translated response to {detected_language}")
            except Exception as e:
                logger.error(f"Error Translating response to {detected_language}: {e}")
                response=response_en
                
        # Add sources information
        sources = []
        for chunk in retrieved_chunks:
            if chunk["path"] not in [s["path"] for s in sources]:
                sources.append({
                    "path": chunk["path"],
                    "title": chunk["title"]
                })
        
        # Use voice interface if requested
        audio_file = None
        if use_voice:
            audio_file = self.voice_interface.generate_audio(response, language=detected_language)
        
        # Add to conversation history
        self.conversation_history.append({
            "query": query,
            "response": response,
            "timestamp": datetime.now().isoformat(),
            "sources": sources,
            "language": detected_language
        })
        
        # Calculate processing time
        processing_time = (datetime.now() - start_time).total_seconds()
        
        return {
            "query": query,
            "response": response,
            "sources": sources,
            "processing_time": processing_time,
            "detected_language": detected_language,
            "audio_file": audio_file,
            "language": detected_language
        }
    
    async def voice_query(self, audio):
        """Handle a voice query from the user"""
        # Process audio input
        query = self.voice_interface.process_audio(audio)
        
        if not query:
            response = {
                "query": "",
                "response": "I didn't hear anything. Please try speaking again.",
                "sources": [],
                "processing_time": 0,
                "detected_language": "en",
                "audio_file": None,
                "language": "en"
            }
            return response["query"], response["response"], ", ".join([s for s in []]), response["audio_file"]
        
        # Process the query
        result = await self.process_query(query, use_voice=True)
        return result["query"], result["response"], ", ".join([s["path"] for s in result["sources"]]), result["audio_file"]
    
    async def video_chat_query(self, message):
        """Handle a text query during video call"""
        if not message.strip():
            return "Please type a message to continue the conversation."
            
        # Process the query as normal text query
        result = await self.process_query(message)
        return result["response"]
    
    def record_feedback(self, query: str, response: str, rating: int, comment: str = "") -> bool:
        """Record user feedback on a response"""
        return self.feedback_system.record_feedback(query, response, rating, comment)
    
    def get_feedback_statistics(self) -> Dict:
        """Get statistics about feedback ratings"""
        return self.feedback_system.get_feedback_statistics()
    
    def get_conversation_history(self) -> List[Dict]:
        """Get the conversation history"""
        return self.conversation_history

# Gradio Interface
def create_interface(assistant):
    """Create a Gradio interface for the Loubby Assistant"""
    
    async def process_text_query(query, voice_output=False):
        """Process a text query and return the response with automatic language detection"""
        if not query.strip():
            return "Please enter a question about the Loubby platform.", "", None, "No query detected"
            
        result = await assistant.process_query(query, use_voice=voice_output)
        detected_language = result["detected_language"]
        language_name = assistant.language_map.get(detected_language, detected_language)
        
        return result["response"], ", ".join([s["path"] for s in result["sources"]]), result["audio_file"], f"Detected language: {language_name}"
    
    async def process_voice_query(audio):
        """Process a voice query and return the response"""
        query, response, sources, audio_file = await assistant.voice_query(audio)
        
        # Detect language for display
        if query:
            detected_language = assistant.language_processor.detect_language(query)
            language_name = assistant.language_map.get(detected_language, detected_language)
            language_info = f"Detected language: {language_name}"
        else:
            language_info = "No speech detected"
            
        return query, response, sources, audio_file, language_info
    
    async def submit_feedback(query, response, rating, comment):
        """Submit feedback for a response"""
        success = assistant.record_feedback(query, response, rating, comment)
        return "Feedback submitted successfully. Thank you!" if success else "Failed to submit feedback. Please try again."
    
    async def show_statistics():
        """Show feedback statistics"""
        stats = assistant.get_feedback_statistics()
        return f"""
        Feedback Statistics:
        - Average Rating: {stats['average_rating']} / 5
        - Total Feedback Count: {stats['total_count']}
        - Rating Distribution: {stats['rating_distribution']}
        """
    
    # Video chat functionality with virtual avatars for cloud environment
    def start_video_call():
        """Start video capture for video chat"""
        success = assistant.video_interface.start_video()
        status = "Video call started with virtual avatars (cloud environment mode)." if success else "Failed to start video call."
        return status
    
    def end_video_call():
        """End video capture for video chat"""
        success = assistant.video_interface.stop_video()
        status = "Video call ended." if success else "Error ending video call."
        return status
    
    async def video_chat_message(message, chat_history):
        """Handle chat messages during video calls"""
        if not message.strip():
            processed_message = "I'm here to help with Loubby platform navigation."
        else:
            processed_message = await assistant.video_chat_query(message)
        
        # Get combined video frame
        response_text, video_frame = assistant.video_interface.process_video_call(message)
        
        # Update chat history
        chat_history = chat_history or []
        chat_history.append((message, processed_message))
        
        return chat_history, gr.update(value=""), video_frame
    
    def video_frame_update():
        """Update the video frame regularly"""
        if not assistant.video_interface.is_running:
            blank_frame = np.zeros((240, 640, 3), dtype=np.uint8)
            cv2.putText(blank_frame, "Start video call to see avatars", (180, 120), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            return blank_frame
            
        return assistant.video_interface.generate_combined_frame()
    
    # Custom CSS for styling
    custom_css = """
    .gradio-container {
        background: linear-gradient(-45deg, #ee7752, #e73c7e, #23a6d5, #23d5ab);
        background_size: 400% 400%;
        animation: gradient 15s ease infinite;
    }
    @keyframes gradient {
        0% { background-position: 0% 50%; }
        50% { background-position: 100% 50%; }
        100% { background-position: 0% 50%; }
    }
    .gradio-interface {
        background: rgba(255, 255, 255, 0.9);
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 4px 30px rgba(0, 0, 0, 0.1);
        backdrop-filter: blur(5px);
        -webkit-backdrop-filter: blur(5px);
    }
    .video-interface {
        background: rgba(255, 255, 255, 0.95);
        border-radius: 15px;
        padding: 15px;
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.15);
    }
    .gradio-feedback {
        background: rgba(245, 245, 245, 0.9);
        border-radius: 10px;
        padding: 15px;
    }
    .video-container {
        display: flex;
        flex-direction: column;
        width: 100%;
    }
    .chat-container {
        display: flex;
        flex-direction: column;
        flex-grow: 1;
        min-height: 300px;
        overflow-y: auto;
        padding: 10px;
        background: rgba(240, 240, 240, 0.7);
        border-radius: 8px;
        margin-top: 15px;
    }
    .language-info {
        background: rgba(255, 255, 255, 0.7);
        padding: 5px 10px;
        border-radius: 5px;
        display: inline-block;
        margin-top: 5px;
        font-size: 0.9em;
        color: #555;
    }
    .cloud-notice {
        background: rgba(255, 200, 0, 0.2);
        border: 1px solid #FFB800;
        padding: 8px;
        border-radius: 5px;
        margin-bottom: 10px;
        font-size: 0.9em;
    }
    """
    
    with gr.Blocks(title="Loubby Navigation Assistant", css=custom_css) as interface:
        gr.Markdown("""
        <div style="text-align: center;">
            <h1 style="color: white;">Loubby AI Navigation Assistant</h1>
            <p style="color: white;">Your personal guide to navigating the Loubby platform</p>
        </div>
        <div class="cloud-notice">
            <strong>Cloud Environment Notice:</strong> Running in cloud mode with virtual video avatars instead of camera input.
        </div>
        """)
        
        with gr.Tabs():
            with gr.Tab("Text Interface", elem_classes="gradio-interface"):
                with gr.Row():
                    with gr.Column(scale=3):
                        text_input = gr.Textbox(label="Your Question", placeholder="How do I upload my resume?", lines=2)
                        
                        language_info = gr.Markdown("Language will be detected automatically", elem_classes="language-info")
                        voice_checkbox = gr.Checkbox(label="Enable Voice Output", value=False)
                        text_submit = gr.Button("Ask", variant="primary")
                    
                    with gr.Column(scale=4):
                        text_output = gr.Textbox(label="Response", lines=8)
                        sources_output = gr.Textbox(label="Sources")
                        audio_output = gr.Audio(label="Voice Response", visible=False)
                
                text_submit.click(
                    process_text_query,
                    inputs=[text_input, voice_checkbox],
                    outputs=[text_output, sources_output, audio_output, language_info]
                )
                
                # Also allow pressing Enter to submit
                text_input.submit(
                    process_text_query,
                    inputs=[text_input, voice_checkbox],
                    outputs=[text_output, sources_output, audio_output, language_info]
                )
                
                voice_checkbox.change(
                    lambda x: gr.Audio.update(visible=x),
                    inputs=voice_checkbox,
                    outputs=audio_output
                )
            
            with gr.Tab("Voice Interface", elem_classes="gradio-interface"):
                with gr.Row():
                    with gr.Column():
                        voice_info = gr.Markdown("Language will be detected automatically", elem_classes="language-info")
                        voice_input = gr.Audio(label="Speak Your Question", type="filepath")
                        voice_button = gr.Button("Submit", variant="primary")
                    with gr.Column():
                        voice_query_display = gr.Textbox(label="Your Question")
                        voice_response = gr.Textbox(label="Response", lines=8)
                        voice_sources = gr.Textbox(label="Sources")
                        voice_audio_output = gr.Audio(label="Voice Response")
                
                voice_button.click(
                    process_voice_query,
                    inputs=[voice_input],
                    outputs=[voice_query_display, voice_response, voice_sources, voice_audio_output, voice_info]
                )
            
            with gr.Tab("Video Communication", elem_classes="video-interface"):
                # Initialize chat history state
                chat_history = gr.State([])
                
                with gr.Row():
                    # Video feed display with combined user and assistant views
                    video_display = gr.Image(label="Video Call", elem_id="video-display")
                    
                    # Controls and chat area
                    with gr.Column():
                        video_call_status = gr.Markdown("Click 'Start Video Call' to begin communicating with the assistant.")
                        
                        # Start and end call buttons
                        with gr.Row():
                            start_button = gr.Button("Start Video Call", variant="primary")
                            end_button = gr.Button("End Video Call", variant="secondary")
                        
                        # Chat display
                        chat_display = gr.Chatbot(label="Conversation", elem_classes="chat-container")
                        
                        # Chat input during video call
                        with gr.Row():
                            video_chat_input = gr.Textbox(
                                label="Type a message", 
                                placeholder="Ask a question about the Loubby platform...",
                                lines=2
                            )
                            video_chat_button = gr.Button("Send", variant="primary")
                
                # Connect everything
                start_button.click(
                    start_video_call,
                    inputs=None,
                    outputs=video_call_status
                )
                
                end_button.click(
                    end_video_call,
                    inputs=None,
                    outputs=video_call_status
                )
                
                # Handle sending messages in video chat
                video_chat_button.click(
                    video_chat_message,
                    inputs=[video_chat_input, chat_history],
                    outputs=[chat_display, video_chat_input, video_display]
                ).then(
                    lambda x: x,
                    inputs=chat_display,
                    outputs=chat_history
                )
                
                # Also allow pressing Enter to send message
                video_chat_input.submit(
                    video_chat_message,
                    inputs=[video_chat_input, chat_history],
                    outputs=[chat_display, video_chat_input, video_display]
                ).then(
                    lambda x: x,
                    inputs=chat_display,
                    outputs=chat_history
                )
                
                # Update video frame every 0.5 seconds
                #gr.on(
                    #"load",
                    #lambda: gr.update(value=video_frame_update()),
                    #None,
                    #video_display,
                    #every=0.5
                #)
            
            with gr.Tab("Feedback", elem_classes="gradio-feedback"):
                with gr.Row():
                    with gr.Column():
                        feedback_query = gr.Textbox(label="Question", lines=2)
                        feedback_response = gr.Textbox(label="Response", lines=4)
                        feedback_rating = gr.Slider(minimum=1, maximum=5, step=1, value=3, label="Rating")
                        feedback_comment = gr.Textbox(label="Comment (Optional)", lines=2)
                        feedback_submit = gr.Button("Submit Feedback", variant="primary")
                        feedback_result = gr.Textbox(label="Result")
                    with gr.Column():
                        stats_button = gr.Button("Show Feedback Statistics", variant="secondary")
                        stats_display = gr.Textbox(label="Statistics")
                
                feedback_submit.click(
                    submit_feedback,
                    inputs=[feedback_query, feedback_response, feedback_rating, feedback_comment],
                    outputs=[feedback_result]
                )
                
                stats_button.click(
                    show_statistics,
                    inputs=None,
                    outputs=[stats_display]
                )
    
    return interface

def run_main():
    """Function to run the main assistant"""
    # Initialize the assistant
    assistant = LoubbyAssistant()
    
    # Create and launch the Gradio interface
    interface = create_interface(assistant)
    interface.launch(share=True)

if __name__ == "__main__":
    run_main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 519)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import os
import re
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import gradio as gr
import logging
import random
from datetime import datetime
import sqlite3
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import speech_recognition as sr
from gtts import gTTS
import uuid
import base64
import tempfile
from io import BytesIO
import threading
import time
import cv2

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
except:
    logger.warning("NLTK download failed. Make sure NLTK is properly installed.")

class LoubbyDocumentation:
    """Class to handle Loubby platform documentation"""
    
    def __init__(self, docs_path="loubby_docs.txt"):
        """Initialize the documentation handler"""
        self.docs_path = docs_path
        self.sections = {}
        self.raw_text = self._load_documentation()
        self.process_documentation()
        
    def _load_documentation(self) -> str:
        """Load documentation from file or create it if it doesn't exist"""
        if not os.path.exists(self.docs_path):
            logger.info(f"Documentation file not found at {self.docs_path}. Creating it...")
            self._create_documentation_file()
        
        with open(self.docs_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def _create_documentation_file(self):
        """Create the documentation file with Loubby platform information"""
        documentation_text = """# Loubby Platform Navigation Guide

## Recruitment Portal

### 1. Profile Setup
#### 1.1 Creating Your Account
To create your account on Loubby's recruitment portal:
1. Navigate to loubby.ai/recruitment
2. Click the "Sign Up" button in the top right corner
3. Enter your email address and create a password
4. Verify your email through the link sent to your inbox
5. Complete your basic profile information (name, contact details)

#### 1.2 Resume Upload
To upload your resume to your Loubby profile:
1. Log in to your account
2. Navigate to "My Profile" from the dashboard menu
3. Scroll to the "Resume" section
4. Click "Upload Resume" button
5. Select your resume file (supported formats: PDF, DOCX, RTF)
6. Click "Save Changes" to confirm upload
7. Verify that your resume appears in the preview section

Tip: For best results, use a PDF format that is ATS-friendly

#### 1.3 Skills Assessment
To complete the skills assessment:
1. From the dashboard, select "Skills Assessment" tab
2. Choose the relevant skill categories for your profile
3. For each category, click "Start Assessment"
4. Complete the assessment questions within the allotted time
5. Submit your answers using the "Submit" button
6. View your results and skill ratings on completion

### 2. Job Search and Application

#### 2.1 Searching for Positions
To search for open positions on Loubby:
1. Click the "Jobs" tab in the main navigation
2. Use the search bar to enter keywords, job titles, or company names
3. Filter results using the sidebar options:
   - Location (remote, on-site, hybrid)
   - Experience level
   - Industry
   - Salary range
   - Posted date
4. Sort results by relevance, date, or salary using the dropdown menu
5. Save searches by clicking "Save This Search" for future reference

#### 2.2 Applying for a Position
To apply for a position:
1. From the job listing, click "Apply Now"
2. Review your profile information for completeness
3. Complete any additional application questions
4. Upload any position-specific documents if requested
5. Review all information for accuracy
6. Click "Submit Application"
7. Confirm submission in the popup window

#### 2.3 Tracking Applications
To track your submitted applications:
1. Navigate to "My Applications" in the dashboard
2. View all applications and their current status
3. Filter by status (submitted, in review, interview scheduled, etc.)
4. Click on any application to view details
5. Check for any action items or requested information

### 3. Interview Process

#### 3.1 Scheduling Interviews
When you receive an interview request:
1. You'll receive an email notification and an in-platform notification
2. Navigate to "My Applications" in the dashboard
3. Find the application with the interview request
4. Click on "Schedule Interview"
5. Select from available time slots in the calendar interface
6. Confirm your selection
7. Add to your personal calendar using the calendar integration options

#### 3.2 Virtual Interview Room
To access the virtual interview room:
1. 15 minutes before your scheduled interview, log in to Loubby
2. Go to "My Applications" in the dashboard
3. Find the relevant application
4. Click "Join Interview" (button becomes active 15 minutes before start time)
5. Test your audio and video in the pre-meeting room
6. Click "Join Now" when ready
7. If you experience technical difficulties, use the support chat in the bottom right

## Employee Portal

### 1. Onboarding

#### 1.1 Accessing the Employee Portal
After accepting a job offer:
1. You'll receive an email with login credentials for the employee portal
2. Navigate to loubby.ai/employee
3. Enter your credentials from the email
4. Set up two-factor authentication if prompted
5. Create a new password that meets security requirements

#### 1.2 Completing Onboarding Documents
To complete necessary onboarding paperwork:
1. From the employee dashboard, select "Onboarding" tab
2. View the list of required documents
3. Click "Complete" next to each document
4. Fill out all required fields
5. Use the e-signature tool where required
6. Click "Submit" for each completed document
7. Track completion progress in the onboarding checklist

#### 1.3 Training Modules
To access and complete training modules:
1. Navigate to "Training" in the main menu
2. View assigned training modules and their deadlines
3. Click on a module to begin
4. Complete all sections of the module
5. Take the assessment quiz at the end
6. Achieve the minimum required score to mark as complete
7. View your training transcript in the "My Learning" section

### 2. Course Synchronization

#### 2.1 Connecting External Courses
To sync external courses with the Loubby platform:
1. From the dashboard, navigate to "Education & Courses"
2. Select "Connect External Course" button
3. Choose from the list of supported learning platforms
4. Log in to your account on the selected platform
5. Authorize the connection when prompted
6. Select which courses to sync with Loubby
7. Click "Confirm" to complete the integration

#### 2.2 Viewing Course Progress
To monitor your progress across all courses:
1. Go to "My Learning" in the main navigation
2. View the dashboard showing all courses and completion percentages
3. Filter by course status, start date, or provider
4. Click on any course to view detailed progress
5. See upcoming assignments and deadlines
6. View instructor feedback on completed assignments

#### 2.3 Submitting Assignments
To submit assignments through Loubby:
1. Navigate to "My Learning" > "Assignments"
2. Find the assignment you want to submit
3. Click "Submit Assignment"
4. Upload required files or enter text submission
5. Add any comments for the instructor
6. Review your submission
7. Click "Submit" to finalize

### 3. Performance Tracking

#### 3.1 Setting Goals
To set and track professional goals:
1. From the employee dashboard, select "Performance" tab
2. Click "My Goals" in the submenu
3. Select "Add New Goal"
4. Define your goal with the SMART criteria
5. Set target date and key milestones
6. Link to relevant skills or competencies
7. Submit for manager approval

#### 3.2 Feedback and Reviews
To access performance feedback:
1. Navigate to "Performance" > "Feedback"
2. View all feedback received from peers and supervisors
3. Filter by date, project, or feedback type
4. For formal reviews, select "Reviews" from the submenu
5. View scheduled and past review cycles
6. Complete self-assessment when prompted
7. Review manager feedback and ratings

## Troubleshooting

### Common Issues

#### Password Reset
If you need to reset your password:
1. On the login page, click "Forgot Password"
2. Enter the email associated your account
3. Check your email for a reset link
4. Click the link and follow instructions to create a new password
5. If you don't receive the email, check spam folder or click "Resend Email"

#### Browser Compatibility
Loubby works best with:
- Chrome (version 90+)
- Firefox (version 88+)
- Safari (version 14+)
- Edge (version 90+)

If experiencing issues:
1. Update your browser to the latest version
2. Clear browser cache and cookies
3. Disable extensions that might interfere
4. Try an alternate supported browser

#### Mobile Access
To access Loubby on mobile devices:
1. Use the responsive web version on any mobile browser
2. Download the Loubby mobile app from App Store or Google Play
3. Log in with the same credentials as the web version
4. Enable notifications for interview alerts
5. Note that some advanced features may require desktop access

### Getting Help

#### Live Support
To access live support:
1. Click the "Help" icon in the bottom right corner
2. Select "Chat with Support"
3. Describe your issue in detail
4. Support is available Monday-Friday, 9am-6pm EST

#### Knowledge Base
To search the help documentation:
1. Click "Help" in the main navigation
2. Use the search bar to find relevant articles
3. Browse by category using the sidebar
4. Rate articles to help improve documentation
"""
        with open(self.docs_path, 'w', encoding='utf-8') as f:
            f.write(documentation_text)
        
        logger.info(f"Documentation file created at {self.docs_path}")
    
    def process_documentation(self):
        """Process the documentation into sections"""
        # Split by main sections using regex
        section_pattern = r'(#+)\s+(.+)'
        current_section = {"title": "Root", "content": "", "level": 0, "subsections": []}
        section_stack = [current_section]
        
        for line in self.raw_text.split('\n'):
            match = re.match(section_pattern, line)
            if match:
                level = len(match.group(1))
                title = match.group(2).strip()
                
                new_section = {"title": title, "content": "", "level": level, "subsections": []}
                
                # Pop from stack until we find the parent section
                while level <= section_stack[-1]["level"]:
                    section_stack.pop()
                
                # Add new section as subsection of parent
                section_stack[-1]["subsections"].append(new_section)
                
                # Push new section to stack
                section_stack.append(new_section)
            else:
                # Add content to current section
                if section_stack[-1]["content"]:
                    section_stack[-1]["content"] += "\n" + line
                else:
                    section_stack[-1]["content"] += line
        
        self.sections = section_stack[0]
        logger.info("Documentation processed successfully")

    def get_flattened_sections(self) -> List[Dict]:
        """Flatten the hierarchical sections into a list with full paths"""
        flattened = []
        
        def traverse(section, path=""):
            current_path = f"{path} > {section['title']}" if path else section['title']
            
            # Add current section
            if section['content'].strip():
                flattened.append({
                    "path": current_path,
                    "title": section['title'],
                    "content": section['content'],
                    "level": section['level']
                })
            
            # Process subsections
            for subsection in section['subsections']:
                traverse(subsection, current_path)
        
        traverse(self.sections)
        return flattened

class DocumentChunker:
    """Class to chunk documentation for efficient retrieval"""
    
    def __init__(self, chunk_size=200, chunk_overlap=50):
        """Initialize the document chunker"""
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
    
    def chunk_document(self, sections: List[Dict]) -> List[Dict]:
        """Split sections into smaller chunks for retrieval"""
        chunked_sections = []
        
        for section in sections:
            # Combine section info with content for context
            full_text = f"{section['path']}\n\n{section['content']}"
            
            # Split text into chunks
            chunks = self.text_splitter.split_text(full_text)
            
            # Create dictionary entries for each chunk
            for i, chunk_text in enumerate(chunks):
                chunked_sections.append({
                    "id": f"{section['path']}_{i}",
                    "path": section['path'],
                    "title": section['title'],
                    "chunk_index": i,
                    "content": chunk_text,
                    "level": section['level']
                })
        
        logger.info(f"Created {len(chunked_sections)} chunks from {len(sections)} sections")
        return chunked_sections

class VectorDatabase:
    """Class to manage vector embeddings and similarity search"""
    
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        """Initialize the vector database"""
        self.model_name = model_name
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
        self.vectorstore = None
        self.chunk_data = None
    
    def create_vectors(self, chunked_sections: List[Dict]):
        """Create vector embeddings for document chunks"""
        # Extract texts and metadata
        texts = [chunk["content"] for chunk in chunked_sections]
        metadatas = [{
            "id": chunk["id"],
            "path": chunk["path"],
            "title": chunk["title"],
            "chunk_index": chunk["chunk_index"],
            "level": chunk["level"]
        } for chunk in chunked_sections]
        
        # Create FAISS index
        self.vectorstore = FAISS.from_texts(
            texts=texts,
            embedding=self.embeddings,
            metadatas=metadatas
        )
        
        # Store the original chunk data for reference
        self.chunk_data = {chunk["id"]: chunk for chunk in chunked_sections}
        
        logger.info(f"Created vector database with {len(texts)} documents")
        return self.vectorstore
    
    def save_vectors(self, path="loubby_vectors"):
        """Save vector database to disk"""
        if self.vectorstore:
            self.vectorstore.save_local(path)
            # Save chunk data separately
            with open(f"{path}_chunks.json", 'w', encoding='utf-8') as f:
                json.dump(self.chunk_data, f)
            logger.info(f"Vector database saved to {path}")
        else:
            logger.error("No vector database to save")
    
    def load_vectors(self, path="loubby_vectors"):
        """Load vector database from disk"""
        if os.path.exists(path):
            try:
                self.vectorstore = FAISS.load_local(path, self.embeddings, allow_dangerous_deserialization=True)
                # Load chunk data
                with open(f"{path}_chunks.json", 'r', encoding='utf-8') as f:
                    self.chunk_data = json.load(f)
                logger.info(f"Vector database loaded from {path}")
                return True
            except Exception as e:
                logger.error(f"Error loading vector database: {e}")
                return False
        else:
            logger.warning(f"No vector database found at {path}")
            return False
    
    def similarity_search(self, query: str, k=3) -> List[Dict]:
        """Search for most similar chunks to the query"""
        if not self.vectorstore:
            logger.error("Vector database not initialized")
            return []
        
        results = self.vectorstore.similarity_search_with_score(query, k=k)
        
        # Format results
        formatted_results = []
        for doc, score in results:
            chunk_id = doc.metadata["id"]
            formatted_results.append({
                "id": chunk_id,
                "path": doc.metadata["path"],
                "title": doc.metadata["title"],
                "content": doc.page_content,
                "similarity": float(score),
                "chunk_index": doc.metadata["chunk_index"],
                "level": doc.metadata["level"]
            })
        
        return formatted_results

class FeedbackSystem:
    """System to collect and use user feedback to improve responses"""
    
    def __init__(self, db_path="loubby_feedback.db"):
        """Initialize the feedback system"""
        self.db_path = db_path
        self._init_database()
    
    def _init_database(self):
        """Initialize the SQLite database for feedback"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Create feedback table if it doesn't exist
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS feedback (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    query TEXT NOT NULL,
                    response TEXT NOT NULL,
                    rating INTEGER NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    user_comment TEXT
                )
            ''')
            
            conn.commit()
            conn.close()
            logger.info(f"Feedback database initialized at {self.db_path}")
        except Exception as e:
            logger.error(f"Error initializing feedback database: {e}")
    
    def record_feedback(self, query: str, response: str, rating: int, user_comment: str = ""):
        """Record user feedback on a response"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute(
                "INSERT INTO feedback (query, response, rating, user_comment) VALUES (?, ?, ?, ?)",
                (query, response, rating, user_comment)
            )
            
            conn.commit()
            conn.close()
            logger.info(f"Feedback recorded: Rating {rating} for query '{query[:30]}...'")
            return True
        except Exception as e:
            logger.error(f"Error recording feedback: {e}")
            return False
    
    def get_historical_feedback(self, limit=100) -> List[Dict]:
        """Get historical feedback data"""
        try:
            conn = sqlite3.connect(self.db_path)
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            cursor.execute(
                "SELECT * FROM feedback ORDER BY timestamp DESC LIMIT ?",
                (limit,)
            )
            
            # Convert to list of dictionaries
            results = [dict(row) for row in cursor.fetchall()]
            
            conn.close()
            return results
        except Exception as e:
            logger.error(f"Error fetching historical feedback: {e}")
            return []
    
    def get_feedback_statistics(self) -> Dict:
        """Get statistics about feedback ratings"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Get average rating
            cursor.execute("SELECT AVG(rating) FROM feedback")
            avg_rating = cursor.fetchone()[0] or 0
            
            # Get rating distribution
            cursor.execute(
                "SELECT rating, COUNT(*) FROM feedback GROUP BY rating ORDER BY rating"
            )
            rating_counts = dict(cursor.fetchall())
            
            # Get total feedback count
            cursor.execute("SELECT COUNT(*) FROM feedback")
            total_count = cursor.fetchone()[0]
            
            conn.close()
            
            return {
                "average_rating": round(avg_rating, 2),
                "rating_distribution": rating_counts,
                "total_count": total_count
            }
        except Exception as e:
            logger.error(f"Error fetching feedback statistics: {e}")
            return {
                "average_rating": 0,
                "rating_distribution": {},
                "total_count": 0
            }

class LanguageProcessor:
    """Class for handling natural language processing tasks"""
    
    def __init__(self, model_path="google/flan-t5-base"):
        """Initialize the language processor"""
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
            logger.info(f"Language processor initialized with model {model_path}")
            
            # Load language detection model
            self.lang_detector = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
            self.language_map = {
                "en": "English",
                "es": "Spanish", 
                "fr": "French",
                "de": "German",
                "zh": "Chinese",
                "ja": "Japanese",
                "ru": "Russian",
                "ar": "Arabic",
                "hi": "Hindi",
                "pt": "Portuguese",
                "it": "Italian",
                "ko": "Korean"
            }
        except Exception as e:
            logger.error(f"Error initializing language processor: {e}")
            self.tokenizer = None
            self.model = None
    
    def detect_language(self, text: str) -> str:
        """Detect the language of input text using a multilingual sentence transformer"""
        if not text.strip():
            return "en"  # Default to English for empty text
        
        try:
            # Get embeddings for the text
            text_embedding = self.lang_detector.encode(text)
            
            # Compare with embeddings of language markers
            language_scores = {}
            for lang, lang_name in self.language_map.items():
                # Create a simple sentence in the target language
                lang_marker = f"This is a sentence in {lang_name}."
                lang_embedding = self.lang_detector.encode(lang_marker)
                
                # Calculate cosine similarity
                similarity = cosine_similarity([text_embedding], [lang_embedding])[0][0]
                language_scores[lang] = similarity
            
            # Return the language with the highest similarity
            detected_lang = max(language_scores.items(), key=lambda x: x[1])[0]
            logger.info(f"Detected language: {detected_lang} with score {language_scores[detected_lang]:.2f}")
            
            return detected_lang
        except Exception as e:
            logger.error(f"Error detecting language: {e}")
            return "en"  # Default to English on error
    
    def translate_text(self, text: str, target_language: str = "en") -> str:
        """Simple translation function (placeholder for actual translation)"""
        # Note: In a real implementation, you would integrate with a translation API
        # Since we're omitting external API dependencies for this cloud environment, 
        # we'll return the original text with a note
        
        detected_language = self.detect_language(text)
        
        # Skip translation if already in target language
        if detected_language == target_language:
            return text
        
        logger.info(f"Translation requested from {detected_language} to {target_language}")
        
        # In a real implementation, you would call a translation service here
        # For now, just return the original text
        return text
    
    def generate_response(self, query: str, retrieved_chunks: List[Dict]) -> str:
        """Generate a comprehensive response based on the retrieved chunks"""
        if not self.model or not self.tokenizer:
            return "Language model not properly initialized."
        
        try:
            # Create a context from retrieved chunks
            context = "\n\n".join([chunk["content"] for chunk in retrieved_chunks])
            
            # Build a prompt
            prompt = f"""
            CONTEXT: {context}
            
            QUERY: {query}
            
            Based on the above context, provide a clear and concise answer to the query about navigating the Loubby platform.
            """
            
            # Generate response
            inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=512,
                min_length=64,
                num_beams=4,
                no_repeat_ngram_size=3,
                early_stopping=True
            )
            
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return response
        except Exception as e:
            logger.error(f"Error generating response: {e}")
            
            # Fallback response method
            most_relevant_chunk = retrieved_chunks[0] if retrieved_chunks else None
            if most_relevant_chunk:
                return f"Based on the Loubby documentation, here's guidance on your question:\n\n{most_relevant_chunk['content']}"
            else:
                return "I'm unable to generate a specific response at the moment. Please try rephrasing your question about the Loubby platform."

class VoiceInterface:
    """Class to handle voice input and output using Gradio components"""
    
    def __init__(self):
        """Initialize the voice interface"""
        logger.info("Voice interface initialized using Gradio components")
    
    def process_audio(self, audio):
        """Process audio input and return text"""
        try:
            if audio is None:
                return ""
            
            # Use Gradio's built-in audio processing
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio) as source:
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data)
                return text
        except Exception as e:
            logger.error(f"Error processing audio: {e}")
            return ""
    
    def generate_audio(self, text, language="en"):
        """Generate audio from text with language support"""
        try:
            if not text.strip():
                return None
            
            tts = gTTS(text=text, lang=language)
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as f:
                tts.save(f.name)
                return f.name
        except Exception as e:
            logger.error(f"Error generating audio: {e}")
            return None

class VideoInterface:
    """Class to handle video communication in cloud environment"""
    
    def __init__(self):
        """Initialize the video interface with dummy video for cloud environments"""
        self.is_running = False
        self.thread = None
        self.frame = None
        self.user_face_detected = False
        self.animation_frame = 0
        self.animation_speed = 0.15
        self.last_animation_time = time.time()
        
        # Create avatars since we can't use real camera
        self.user_avatar = self._create_user_avatar()
        self.assistant_avatar = self._create_assistant_avatar()
        
        logger.info("Video interface initialized with virtual avatars for cloud environment")
    
    def _create_user_avatar(self):
        """Create a simple avatar for the user in cloud environments"""
        avatar = np.zeros((240, 320, 3), dtype=np.uint8)
        # Set background color
        avatar[:, :] = (50, 50, 50)
        # Draw a simple circle face
        cv2.circle(avatar, (160, 120), 80, (100, 100, 220), -1)
        # Draw eyes
        cv2.circle(avatar, (130, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (190, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (130, 100), 7, (0, 0, 0), -1)
        cv2.circle(avatar, (190, 100), 7, (0, 0, 0), -1)
        # Draw mouth
        cv2.ellipse(avatar, (160, 140), (40, 20), 0, 0, 180, (0, 0, 0), 3)
        
        # Add text label
        cv2.putText(avatar, "User (virtual)", (80, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(avatar, "Cloud Environment", (70, 230), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        
        return avatar
    
    def _create_assistant_avatar(self):
        """Create a simple avatar for the assistant"""
        avatar = np.zeros((240, 320, 3), dtype=np.uint8)
        # Set background color
        avatar[:, :] = (30, 30, 30)
        # Draw a simple circle face
        cv2.circle(avatar, (160, 120), 80, (70, 130, 180), -1)
        # Draw eyes
        cv2.circle(avatar, (130, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (190, 100), 15, (255, 255, 255), -1)
        cv2.circle(avatar, (130, 100), 7, (0, 0, 0), -1)
        cv2.circle(avatar, (190, 100), 7, (0, 0, 0), -1)
        # Draw mouth (will be animated)
        cv2.ellipse(avatar, (160, 140), (40, 15), 0, 0, 180, (0, 0, 0), 3)
        
        # Add text label
        cv2.putText(avatar, "Loubby Assistant", (100, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        return avatar
    
    def _update_avatar_animation(self):
        """Update the avatar's animation"""
        current_time = time.time()
        if current_time - self.last_animation_time > self.animation_speed:
            # Create a copy of the base avatar
            animated_avatar = self.assistant_avatar.copy()
            
            # Animate the mouth based on speaking state
            mouth_height = 10 + 10 * np.sin(self.animation_frame)
            cv2.ellipse(animated_avatar, (160, 140), (40, int(mouth_height)), 0, 0, 180, (0, 0, 0), 3)
            
            self.animation_frame += 0.5
            self.last_animation_time = current_time
            
            return animated_avatar
        return self.assistant_avatar.copy()
    
    def start_video(self):
        """Start the virtual video session"""
        if self.is_running:
            return True
        
        self.is_running = True
        
        # Start animation thread
        self.thread = threading.Thread(target=self._animate_avatars)
        self.thread.daemon = True
        self.thread.start()
        
        logger.info("Virtual video avatars started for cloud environment")
        return True
    
    def stop_video(self):
        """Stop the virtual video session"""
        self.is_running = False
        
        if self.thread:
            self.thread.join(timeout=1.0)
            self.thread = None
        
        logger.info("Virtual video avatars stopped")
        return True
    
    def _animate_avatars(self):
        """Animate the avatars in a continuous loop"""
        while self.is_running:
            # Update the assistant avatar animation
            self._update_avatar_animation()
            time.sleep(0.05)
    
    def get_user_frame(self):
        """Get the user's avatar frame"""
        return self.user_avatar
    
    def get_assistant_frame(self):
        """Get the assistant's animated avatar frame"""
        return self._update_avatar_animation()
    
    def generate_combined_frame(self):
        """Generate a combined frame with user and assistant avatars"""
        user_frame = self.get_user_frame()
        assistant_frame = self.get_assistant_frame()
        
        # Create a combined frame (side by side)
        combined = np.zeros((240, 640, 3), dtype=np.uint8)
        combined[:, :320] = user_frame
        combined[:, 320:] = assistant_frame
        
        # Add a separating line
        cv2.line(combined, (320, 0), (320, 240), (100, 100, 100), 2)
        
        # Add a status indicator showing this is a cloud environment
        cv2.rectangle(combined, (180, 10), (460, 30), (0, 0, 100), -1)
        cv2.putText(combined, "Cloud Environment - Virtual Video", (190, 25), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return combined
    
    def process_video_call(self, text_message):
        """Process a simulated video call with the assistant"""
        if not self.is_running:
            return "Please start the video call first.", self._create_assistant_avatar()
        
        # Generate a combined frame with user and assistant
        combined_frame = self.generate_combined_frame()
        
        if not text_message:
            return "I'm here and ready to help! Type a message to ask about the Loubby platform.", combined_frame
        
        return f"Processing: '{text_message}'", combined_frame

class LoubbyAssistant:
    """Main assistant class that integrates all components"""
    
    def __init__(self, load_existing=True):
        """Initialize the Loubby Assistant"""
        self.documentation = LoubbyDocumentation()
        self.chunker = DocumentChunker(chunk_size=300, chunk_overlap=100)
        self.vector_db = VectorDatabase()
        self.feedback_system = FeedbackSystem()
        self.language_processor = LanguageProcessor()
        self.voice_interface = VoiceInterface()
        self.video_interface = VideoInterface()
        
        # Initialize vector database
        if load_existing and self.vector_db.load_vectors():
            logger.info("Using existing vector database")
        else:
            logger.info("Creating new vector database")
            sections = self.documentation.get_flattened_sections()
            chunks = self.chunker.chunk_document(sections)
            self.vector_db.create_vectors(chunks)
            self.vector_db.save_vectors()
        
        # Track conversation history
        self.conversation_history = []
        
        # Map of language codes to full names for internal use
        self.language_map = {
            "en": "English",
            "es": "Spanish", 
            "fr": "French",
            "de": "German",
            "zh": "Chinese",
            "ja": "Japanese",
            "ru": "Russian",
            "ar": "Arabic",
            "hi": "Hindi",
            "pt": "Portuguese",
            "it": "Italian",
            "ko": "Korean"
        }
        
        logger.info("Loubby Assistant initialized and ready")
    
    async def process_query(self, query: str, use_voice=False) -> Dict:
        """Process a user query and generate a response with language detection"""
        start_time = datetime.now()
        
        # Detect language (using our improved detection method)
        detected_language = self.language_processor.detect_language(query)
        logger.info(f"Detected language: {detected_language}")
        
        translated_query = query
        
        # Translate to English for processing if not already in English
        if detected_language != "en":
            translated_query = self.language_processor.translate_text(query, target_language="en")
            logger.info(f"Translated query from {detected_language} to English: {translated_query}")
        
        # Search for relevant chunks
        retrieved_chunks = self.vector_db.similarity_search(translated_query, k=3)
        
        # Generate response in English
        response_en = ""
        if retrieved_chunks:
            response_en = self.language_processor.generate_response(translated_query, retrieved_chunks)
        else:
            response_en = "I'm sorry, I couldn't find specific information about that in the Loubby documentation. Could you please rephrase your question about using the Loubby platform?"
        
        # Translate response back to original language if needed
        response = response_en
        if detected_language != "en":
            try:
               response = self.language_processor.translate_text(response_en, target_language=detected_language)
               logger.info(f"Translated response to {detected_language}")
            except Exception as e:
                logger.error(f"Error Translating response to {detected_language}: {e}")
                response=response_en
                
        # Add sources information
        sources = []
        for chunk in retrieved_chunks:
            if chunk["path"] not in [s["path"] for s in sources]:
                sources.append({
                    "path": chunk["path"],
                    "title": chunk["title"]
                })
        
        # Use voice interface if requested
        audio_file = None
        if use_voice:
            audio_file = self.voice_interface.generate_audio(response, language=detected_language)
        
        # Add to conversation history
        self.conversation_history.append({
            "query": query,
            "response": response,
            "timestamp": datetime.now().isoformat(),
            "sources": sources,
            "language": detected_language
        })
        
        # Calculate processing time
        processing_time = (datetime.now() - start_time).total_seconds()
        
        return {
            "query": query,
            "response": response,
            "sources": sources,
            "processing_time": processing_time,
            "detected_language": detected_language,
            "audio_file": audio_file,
            "language": detected_language
        }
    
    async def voice_query(self, audio):
        """Handle a voice query from the user"""
        # Process audio input
        query = self.voice_interface.process_audio(audio)
        
        if not query:
            response = {
                "query": "",
                "response": "I didn't hear anything. Please try speaking again.",
                "sources": [],
                "processing_time": 0,
                "detected_language": "en",
                "audio_file": None,
                "language": "en"
            }
            return response["query"], response["response"], ", ".join([s for s in []]), response["audio_file"]
        
        # Process the query
        result = await self.process_query(query, use_voice=True)
        return result["query"], result["response"], ", ".join([s["path"] for s in result["sources"]]), result["audio_file"]
    
    async def video_chat_query(self, message):
        """Handle a text query during video call"""
        if not message.strip():
            return "Please type a message to continue the conversation."
            
        # Process the query as normal text query
        result = await self.process_query(message)
        return result["response"]
    
    def record_feedback(self, query: str, response: str, rating: int, comment: str = "") -> bool:
        """Record user feedback on a response"""
        return self.feedback_system.record_feedback(query, response, rating, comment)
    
    def get_feedback_statistics(self) -> Dict:
        """Get statistics about feedback ratings"""
        return self.feedback_system.get_feedback_statistics()
    
    def get_conversation_history(self) -> List[Dict]:
        """Get the conversation history"""
        return self.conversation_history

# Gradio Interface
def create_interface(assistant):
    """Create a Gradio interface for the Loubby Assistant"""
    
    async def process_text_query(query, voice_output=False):
        """Process a text query and return the response with automatic language detection"""
        if not query.strip():
            return "Please enter a question about the Loubby platform.", "", None, "No query detected"
            
        result = await assistant.process_query(query, use_voice=voice_output)
        detected_language = result["detected_language"]
        language_name = assistant.language_map.get(detected_language, detected_language)
        
        return result["response"], ", ".join([s["path"] for s in result["sources"]]), result["audio_file"], f"Detected language: {language_name}"
    
    async def process_voice_query(audio):
        """Process a voice query and return the response"""
        query, response, sources, audio_file = await assistant.voice_query(audio)
        
        # Detect language for display
        if query:
            detected_language = assistant.language_processor.detect_language(query)
            language_name = assistant.language_map.get(detected_language, detected_language)
            language_info = f"Detected language: {language_name}"
        else:
            language_info = "No speech detected"
            
        return query, response, sources, audio_file, language_info
    
    async def submit_feedback(query, response, rating, comment):
        """Submit feedback for a response"""
        success = assistant.record_feedback(query, response, rating, comment)
        return "Feedback submitted successfully. Thank you!" if success else "Failed to submit feedback. Please try again."
    
    async def show_statistics():
        """Show feedback statistics"""
        stats = assistant.get_feedback_statistics()
        return f"""
        Feedback Statistics:
        - Average Rating: {stats['average_rating']} / 5
        - Total Feedback Count: {stats['total_count']}
        - Rating Distribution: {stats['rating_distribution']}
        """
    
    # Video chat functionality with virtual avatars for cloud environment
    def start_video_call():
        """Start video capture for video chat"""
        success = assistant.video_interface.start_video()
        status = "Video call started with virtual avatars (cloud environment mode)." if success else "Failed to start video call."
        return status
    
    def end_video_call():
        """End video capture for video chat"""
        success = assistant.video_interface.stop_video()
        status = "Video call ended." if success else "Error ending video call."
        return status
    
    async def video_chat_message(message, chat_history):
        """Handle chat messages during video calls"""
        if not message.strip():
            processed_message = "I'm here to help with Loubby platform navigation."
        else:
            processed_message = await assistant.video_chat_query(message)
        
        # Get combined video frame
        response_text, video_frame = assistant.video_interface.process_video_call(message)
        
        # Update chat history
        chat_history = chat_history or []
        chat_history.append((message, processed_message))
        
        return chat_history, gr.update(value=""), video_frame
    
    def video_frame_update():
        """Update the video frame regularly"""
        if not assistant.video_interface.is_running:
            blank_frame = np.zeros((240, 640, 3), dtype=np.uint8)
            cv2.putText(blank_frame, "Start video call to see avatars", (180, 120), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            return blank_frame
            
        return assistant.video_interface.generate_combined_frame()
    
    # Custom CSS for styling
    custom_css = """
    .gradio-container {
        background: linear-gradient(-45deg, #ee7752, #e73c7e, #23a6d5, #23d5ab);
        background_size: 400% 400%;
        animation: gradient 15s ease infinite;
    }
    @keyframes gradient {
        0% { background-position: 0% 50%; }
        50% { background-position: 100% 50%; }
        100% { background-position: 0% 50%; }
    }
    .gradio-interface {
        background: rgba(255, 255, 255, 0.9);
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 4px 30px rgba(0, 0, 0, 0.1);
        backdrop-filter: blur(5px);
        -webkit-backdrop-filter: blur(5px);
    }
    .video-interface {
        background: rgba(255, 255, 255, 0.95);
        border-radius: 15px;
        padding: 15px;
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.15);
    }
    .gradio-feedback {
        background: rgba(245, 245, 245, 0.9);
        border-radius: 10px;
        padding: 15px;
    }
    .video-container {
        display: flex;
        flex-direction: column;
        width: 100%;
    }
    .chat-container {
        display: flex;
        flex-direction: column;
        flex-grow: 1;
        min-height: 300px;
        overflow-y: auto;
        padding: 10px;
        background: rgba(240, 240, 240, 0.7);
        border-radius: 8px;
        margin-top: 15px;
    }
    .language-info {
        background: rgba(255, 255, 255, 0.7);
        padding: 5px 10px;
        border-radius: 5px;
        display: inline-block;
        margin-top: 5px;
        font-size: 0.9em;
        color: #555;
    }
    .cloud-notice {
        background: rgba(255, 200, 0, 0.2);
        border: 1px solid #FFB800;
        padding: 8px;
        border-radius: 5px;
        margin-bottom: 10px;
        font-size: 0.9em;
    }
    """
    
    with gr.Blocks(title="Loubby Navigation Assistant", css=custom_css) as interface:
        gr.Markdown("""
        <div style="text-align: center;">
            <h1 style="color: white;">Loubby AI Navigation Assistant</h1>
            <p style="color: white;">Your personal guide to navigating the Loubby platform</p>
        </div>
        <div class="cloud-notice">
            <strong>Cloud Environment Notice:</strong> Running in cloud mode with virtual video avatars instead of camera input.
        </div>
        """)
        
        with gr.Tabs():
            with gr.Tab("Text Interface", elem_classes="gradio-interface"):
                with gr.Row():
                    with gr.Column(scale=3):
                        text_input = gr.Textbox(label="Your Question", placeholder="How do I upload my resume?", lines=2)
                        
                        language_info = gr.Markdown("Language will be detected automatically", elem_classes="language-info")
                        voice_checkbox = gr.Checkbox(label="Enable Voice Output", value=False)
                        text_submit = gr.Button("Ask", variant="primary")
                    
                    with gr.Column(scale=4):
                        text_output = gr.Textbox(label="Response", lines=8)
                        sources_output = gr.Textbox(label="Sources")
                        audio_output = gr.Audio(label="Voice Response", visible=False)
                
                text_submit.click(
                    process_text_query,
                    inputs=[text_input, voice_checkbox],
                    outputs=[text_output, sources_output, audio_output, language_info]
                )
                
                # Also allow pressing Enter to submit
                text_input.submit(
                    process_text_query,
                    inputs=[text_input, voice_checkbox],
                    outputs=[text_output, sources_output, audio_output, language_info]
                )
                
                voice_checkbox.change(
                    lambda x: gr.Audio.update(visible=x),
                    inputs=voice_checkbox,
                    outputs=audio_output
                )
            
            with gr.Tab("Voice Interface", elem_classes="gradio-interface"):
                with gr.Row():
                    with gr.Column():
                        voice_info = gr.Markdown("Language will be detected automatically", elem_classes="language-info")
                        voice_input = gr.Audio(label="Speak Your Question", type="filepath")
                        voice_button = gr.Button("Submit", variant="primary")
                    with gr.Column():
                        voice_query_display = gr.Textbox(label="Your Question")
                        voice_response = gr.Textbox(label="Response", lines=8)
                        voice_sources = gr.Textbox(label="Sources")
                        voice_audio_output = gr.Audio(label="Voice Response")
                
                voice_button.click(
                    process_voice_query,
                    inputs=[voice_input],
                    outputs=[voice_query_display, voice_response, voice_sources, voice_audio_output, voice_info]
                )
            
            with gr.Tab("Video Communication", elem_classes="video-interface"):
                # Initialize chat history state
                chat_history = gr.State([])
                
                with gr.Row():
                    # Video feed display with combined user and assistant views
                    video_display = gr.Image(label="Video Call", elem_id="video-display")
                    
                    # Controls and chat area
                    with gr.Column():
                        video_call_status = gr.Markdown("Click 'Start Video Call' to begin communicating with the assistant.")
                        
                        # Start and end call buttons
                        with gr.Row():
                            start_button = gr.Button("Start Video Call", variant="primary")
                            end_button = gr.Button("End Video Call", variant="secondary")
                        
                        # Chat display
                        chat_display = gr.Chatbot(label="Conversation", elem_classes="chat-container")
                        
                        # Chat input during video call
                        with gr.Row():
                            video_chat_input = gr.Textbox(
                                label="Type a message", 
                                placeholder="Ask a question about the Loubby platform...",
                                lines=2
                            )
                            video_chat_button = gr.Button("Send", variant="primary")
                
                # Connect everything
                start_button.click(
                    start_video_call,
                    inputs=None,
                    outputs=video_call_status
                )
                
                end_button.click(
                    end_video_call,
                    inputs=None,
                    outputs=video_call_status
                )
                
                # Handle sending messages in video chat
                video_chat_button.click(
                    video_chat_message,
                    inputs=[video_chat_input, chat_history],
                    outputs=[chat_display, video_chat_input, video_display]
                ).then(
                    lambda x: x,
                    inputs=chat_display,
                    outputs=chat_history
                )
                
                # Also allow pressing Enter to send message
                video_chat_input.submit(
                    video_chat_message,
                    inputs=[video_chat_input, chat_history],
                    outputs=[chat_display, video_chat_input, video_display]
                ).then(
                    lambda x: x,
                    inputs=chat_display,
                    outputs=chat_history
                )
                
                # Update video frame every 0.5 seconds
                #gr.on(
                    #"load",
                    #lambda: gr.update(value=video_frame_update()),
                    #None,
                    #video_display,
                    #every=0.5
                #)
            
            with gr.Tab("Feedback", elem_classes="gradio-feedback"):
                with gr.Row():
                    with gr.Column():
                        feedback_query = gr.Textbox(label="Question", lines=2)
                        feedback_response = gr.Textbox(label="Response", lines=4)
                        feedback_rating = gr.Slider(minimum=1, maximum=5, step=1, value=3, label="Rating")
                        feedback_comment = gr.Textbox(label="Comment (Optional)", lines=2)
                        feedback_submit = gr.Button("Submit Feedback", variant="primary")
                        feedback_result = gr.Textbox(label="Result")
                    with gr.Column():
                        stats_button = gr.Button("Show Feedback Statistics", variant="secondary")
                        stats_display = gr.Textbox(label="Statistics")
                
                feedback_submit.click(
                    submit_feedback,
                    inputs=[feedback_query, feedback_response, feedback_rating, feedback_comment],
                    outputs=[feedback_result]
                )
                
                stats_button.click(
                    show_statistics,
                    inputs=None,
                    outputs=[stats_display]
                )
    
    return interface

def run_main():
    """Function to run the main assistant"""
    # Initialize the assistant
    assistant = LoubbyAssistant()
    
    # Create and launch the Gradio interface
    interface = create_interface(assistant)
    interface.launch(share=True)

if __name__ == "__main__":
    run_main()

2025-03-06 00:45:22.519042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 00:45:22.533972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741221922.553807  181441 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741221922.559885  181441 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 00:45:22.579163: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-06 00:53:51,105 - INFO - Detected language: de with score 0.30
2025-03-06 00:53:51,106 - INFO - Translated response to de
